In [1]:
import pymongo

In [111]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)

In [3]:
import pprint

In [201]:
import datetime
from datetime import date
import time
from datetime import timedelta, date

In [5]:
uri = "mongodb://weather-report-mongodb:2YCR1nxsdoHtvyFJRYwEESBAbkDDmC9QGtnh3cK9zarK5uvWDFCWLpy3jA0vErtr6muGmVpwWz5mQ3TxeEKOOw==@weather-report-mongodb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@weather-report-mongodb@&retrywrites=false"
client = pymongo.MongoClient(uri)
wrdb = client["g_cal_data"]
cont1 = wrdb["events"]

In [91]:
# data model for calendar data
# date | start | end

event_list = []
for event in cont1.find():
    start_date_str = event['start']['dateTime']
    start_date_time_obj = datetime.datetime.strptime(start_date_str, '%Y-%m-%dT%H:%M:%S%z')
    
    date_value = start_date_time_obj.date()
    start_time_value = start_date_time_obj.time()
    
    end_date_str = event['end']['dateTime']
    end_date_time_obj = datetime.datetime.strptime(end_date_str, '%Y-%m-%dT%H:%M:%S%z')
    end_time_value = end_date_time_obj.time()
    
    event_list.append(
        pd.DataFrame(
            data={
                'date':[date_value],
                'start':start_time_value,
                'end':end_time_value,
                'start_datetime':(str(date_value) + ' ' + str(start_time_value)),
                'end_datetime':(str(date_value) + ' ' + str(end_time_value))
            },
            dtype=str
        )
    )

event_df = pd.concat(event_list)
event_df['start_datetime'] = pd.to_datetime(event_df['start_datetime'])
event_df['end_datetime'] = pd.to_datetime(event_df['end_datetime'])

defining the optimal merged model

I think it should look something like this: metrics every 15 mins

date | time (15 min blocks) | is_busy | is_raining | is_sunlight | is_opt_temp | should_workout

In [57]:
# get the current date in yyyy-mm-dd format
current_date = date.today()

# get the starting time represented as an integer
start_time = 900

# initialize list that holds pd records
timestamp_pd_list = []

# iterate through and add increments of 15 mins
for num in range(0,96):
    timestamp_pd_list.append(
        pd.DataFrame(
            data={
                'datetime':[str(current_date) + ' ' + time.strftime('%H:%M:%S', time.gmtime(start_time))],
                'is_available':1
            }
        )        
    )
#     timestamp_pd_list.append(str(current_date) + ' ' + time.strftime('%H:%M:%S', time.gmtime(start_time)))
    
    start_time += 900
timestamp_df = pd.concat(timestamp_pd_list)
timestamp_df['datetime'] = pd.to_datetime(timestamp_df['datetime'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 0
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   datetime      96 non-null     datetime64[ns]
 1   is_available  96 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 2.2 KB


In [101]:
curr_date_events = event_df[event_df['date']==str(current_date)]

In [80]:
timestamp_df

,datetime,is_available
0,2020-09-05 00:15:00,1
0,2020-09-05 00:30:00,1
0,2020-09-05 00:45:00,1
0,2020-09-05 01:00:00,1
0,2020-09-05 01:15:00,1
...,...,...
0,2020-09-05 23:00:00,1
0,2020-09-05 23:15:00,1
0,2020-09-05 23:30:00,1
0,2020-09-05 23:45:00,1


In [102]:
# get unique start-end times for the given day
unavail_times = curr_date_events[['start_datetime','end_datetime']].drop_duplicates()
unavail_times

,start_datetime,end_datetime
0,2020-09-05 12:30:00,2020-09-05 13:30:00


In [106]:
#unavail_times_lol = unavail_times.to_numpy().tolist()
unavail_times_lol = list(unavail_times.to_records(index=False))
unavail_times_lol

[('2020-09-05T12:30:00.000000000', '2020-09-05T13:30:00.000000000')]

In [108]:
test_np_time_val = unavail_times_lol[0][0]

In [110]:
timestamp_df_test = timestamp_df
timestamp_df_test['is_avail_test'] = [0 if x <= test_np_time_val else 1 for x in timestamp_df_test['datetime']] 
timestamp_df_test

,datetime,is_available,is_avail_test
0,2020-09-05 00:15:00,1,0
0,2020-09-05 00:30:00,1,0
0,2020-09-05 00:45:00,1,0
0,2020-09-05 01:00:00,1,0
0,2020-09-05 01:15:00,1,0
...,...,...,...
0,2020-09-05 23:00:00,1,1
0,2020-09-05 23:15:00,1,1
0,2020-09-05 23:30:00,1,1
0,2020-09-05 23:45:00,1,1


In [112]:
for event in unavail_times_lol:
    start = event[0]
    end = event[1]
    timestamp_df['is_available'] = [0 if x >= start and x <= end else 1 for x in timestamp_df['datetime']]
timestamp_df

,datetime,is_available,is_avail_test
0,2020-09-05 00:15:00,1,0
0,2020-09-05 00:30:00,1,0
0,2020-09-05 00:45:00,1,0
0,2020-09-05 01:00:00,1,0
0,2020-09-05 01:15:00,1,0
0,2020-09-05 01:30:00,1,0
0,2020-09-05 01:45:00,1,0
0,2020-09-05 02:00:00,1,0
0,2020-09-05 02:15:00,1,0
0,2020-09-05 02:30:00,1,0


In [113]:
# get weather report data
wrdb = client["weather_report"]
cont1 = wrdb["daily_temps"]

In [173]:
we_ro_test = cont1.find_one({'date':'2020-09-05'})

In [119]:
we_ro_test

{'_id': ObjectId('5f5183825fabbeb2bf8851e9'),
 'date': '2020-09-06',
 'sunrise': '2020-09-06T10:45:09+00:00',
 'sunset': '2020-09-06T23:30:10+00:00',
 'solar_noon': '2020-09-06T17:07:39+00:00',
 'periods': [{'number': 0,
   'name': '',
   'startTime': '2020-09-06T00:00:00-04:00',
   'endTime': '2020-09-06T01:00:00-04:00',
   'isDaytime': False,
   'temperature': 64,
   'temperatureUnit': 'F',
   'temperatureTrend': None,
   'windSpeed': '1 mph',
   'windDirection': 'N',
   'icon': 'https://api.weather.gov/icons/land/night/skc?size=small',
   'shortForecast': 'Clear',
   'detailedForecast': ''},
  {'number': 1,
   'name': '',
   'startTime': '2020-09-06T01:00:00-04:00',
   'endTime': '2020-09-06T02:00:00-04:00',
   'isDaytime': False,
   'temperature': 63,
   'temperatureUnit': 'F',
   'temperatureTrend': None,
   'windSpeed': '1 mph',
   'windDirection': 'NW',
   'icon': 'https://api.weather.gov/icons/land/night/skc?size=small',
   'shortForecast': 'Clear',
   'detailedForecast': ''},


In [185]:
# transform raw json into data model
# date | time in hours | shortForecast | temperature
# then to
# date | time (15 min blocks) | shortForecast | temperature

periods = we_ro_test['periods']

In [186]:
weather_hours_list = []
for period in periods:
    weather_hours_list.append(
        pd.DataFrame(
            data={
                'date':[current_date],
                'time':period['startTime'][:19],
                'short_forcast':period['shortForecast'],
                'temperature':period['temperature']
            }
        )
    )
weather_hours_df = pd.concat(weather_hours_list)
weather_hours_df

,date,time,short_forcast,temperature
0,2020-09-07,2020-09-05T00:00:00,Partly Cloudy,72
0,2020-09-07,2020-09-05T01:00:00,Partly Cloudy,71
0,2020-09-07,2020-09-05T02:00:00,Partly Cloudy,71
0,2020-09-07,2020-09-05T03:00:00,Partly Cloudy,70
0,2020-09-07,2020-09-05T04:00:00,Partly Cloudy,69
0,2020-09-07,2020-09-05T05:00:00,Partly Cloudy,68
0,2020-09-07,2020-09-05T06:00:00,Mostly Sunny,68
0,2020-09-07,2020-09-05T07:00:00,Mostly Sunny,68
0,2020-09-07,2020-09-05T08:00:00,Mostly Sunny,69
0,2020-09-07,2020-09-05T09:00:00,Sunny,70


In [187]:
weather_hours_df['time']=pd.to_datetime(weather_hours_df['time'])
weather_hours_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 0
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           24 non-null     object        
 1   time           24 non-null     datetime64[ns]
 2   short_forcast  24 non-null     object        
 3   temperature    24 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 960.0+ bytes


In [188]:
weather_hours_df.head()


,date,time,short_forcast,temperature
0,2020-09-07,2020-09-05 00:00:00,Partly Cloudy,72
0,2020-09-07,2020-09-05 01:00:00,Partly Cloudy,71
0,2020-09-07,2020-09-05 02:00:00,Partly Cloudy,71
0,2020-09-07,2020-09-05 03:00:00,Partly Cloudy,70
0,2020-09-07,2020-09-05 04:00:00,Partly Cloudy,69


In [189]:
weather_hours_df['time']+pd.Timedelta('15 minutes')

0   2020-09-05 00:15:00
0   2020-09-05 01:15:00
0   2020-09-05 02:15:00
0   2020-09-05 03:15:00
0   2020-09-05 04:15:00
0   2020-09-05 05:15:00
0   2020-09-05 06:15:00
0   2020-09-05 07:15:00
0   2020-09-05 08:15:00
0   2020-09-05 09:15:00
0   2020-09-05 10:15:00
0   2020-09-05 11:15:00
0   2020-09-05 12:15:00
0   2020-09-05 13:15:00
0   2020-09-05 14:15:00
0   2020-09-05 15:15:00
0   2020-09-05 16:15:00
0   2020-09-05 17:15:00
0   2020-09-05 18:15:00
0   2020-09-05 19:15:00
0   2020-09-05 20:15:00
0   2020-09-05 21:15:00
0   2020-09-05 22:15:00
0   2020-09-05 23:15:00
Name: time, dtype: datetime64[ns]

In [202]:
# get the current date in yyyy-mm-dd format
current_date = date.today() - timedelta(days=2)

# get the starting time represented as an integer
start_time = 900

# initialize list that holds pd records
timestamp_pd_list_w = []

# iterate through and add increments of 15 mins
for num in range(0,96):
    timestamp_pd_list_w.append(
        pd.DataFrame(
            data={
                'datetime':[str(current_date) + ' ' + time.strftime('%H:%M:%S', time.gmtime(start_time))]
            }
        )        
    )
#     timestamp_pd_list.append(str(current_date) + ' ' + time.strftime('%H:%M:%S', time.gmtime(start_time)))
    
    start_time += 900
timestamp_df_w = pd.concat(timestamp_pd_list_w)
timestamp_df_w['datetime'] = pd.to_datetime(timestamp_df_w['datetime'])

In [203]:
# round down to the nearest hour
timestamp_df_w['nearest_hour'] = timestamp_df_w['datetime'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 60)))

In [204]:
timestamp_df_w.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 0
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   datetime      96 non-null     datetime64[ns]
 1   nearest_hour  96 non-null     datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 2.2 KB


In [205]:
# try to join the two dataframes
weather_data_pre_dropped = timestamp_df_w.merge(weather_hours_df, left_on='nearest_hour', right_on='time')

In [206]:
weather_data_pre_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   datetime       96 non-null     datetime64[ns]
 1   nearest_hour   96 non-null     datetime64[ns]
 2   date           96 non-null     object        
 3   time           96 non-null     datetime64[ns]
 4   short_forcast  96 non-null     object        
 5   temperature    96 non-null     int64         
dtypes: datetime64[ns](3), int64(1), object(2)
memory usage: 5.2+ KB


In [207]:
weather_data_pre_dropped.head()

,datetime,nearest_hour,date,time,short_forcast,temperature
0,2020-09-05 00:15:00,2020-09-05 00:00:00,2020-09-07,2020-09-05 00:00:00,Partly Cloudy,72
1,2020-09-05 00:30:00,2020-09-05 00:00:00,2020-09-07,2020-09-05 00:00:00,Partly Cloudy,72
2,2020-09-05 00:45:00,2020-09-05 00:00:00,2020-09-07,2020-09-05 00:00:00,Partly Cloudy,72
3,2020-09-05 00:00:00,2020-09-05 00:00:00,2020-09-07,2020-09-05 00:00:00,Partly Cloudy,72
4,2020-09-05 01:00:00,2020-09-05 01:00:00,2020-09-07,2020-09-05 01:00:00,Partly Cloudy,71


In [208]:
weather_data_ready = weather_data_pre_dropped.drop(columns=['nearest_hour', 'date','time'])
weather_data_ready.head()

,datetime,short_forcast,temperature
0,2020-09-05 00:15:00,Partly Cloudy,72
1,2020-09-05 00:30:00,Partly Cloudy,72
2,2020-09-05 00:45:00,Partly Cloudy,72
3,2020-09-05 00:00:00,Partly Cloudy,72
4,2020-09-05 01:00:00,Partly Cloudy,71


In [209]:
weather_data_ready.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   datetime       96 non-null     datetime64[ns]
 1   short_forcast  96 non-null     object        
 2   temperature    96 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 3.0+ KB


In [210]:
#timestamp_df = timestamp_df.drop(columns=['is_avail_test'])
timestamp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 0
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   datetime      96 non-null     datetime64[ns]
 1   is_available  96 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 2.2 KB


In [172]:
timestamp_df.head()

,datetime,is_available
0,2020-09-05 00:15:00,1
0,2020-09-05 00:30:00,1
0,2020-09-05 00:45:00,1
0,2020-09-05 01:00:00,1
0,2020-09-05 01:15:00,1


In [211]:
# join availability with weather data
training_data_to_edit = weather_data_ready.merge(timestamp_df, left_on='datetime', right_on='datetime')
training_data_to_edit.head()

,datetime,short_forcast,temperature,is_available
0,2020-09-05 00:15:00,Partly Cloudy,72,1
1,2020-09-05 00:30:00,Partly Cloudy,72,1
2,2020-09-05 00:45:00,Partly Cloudy,72,1
3,2020-09-05 00:00:00,Partly Cloudy,72,1
4,2020-09-05 01:00:00,Partly Cloudy,71,1


## Rerun everything by generating training data for all available data

In [1]:
import pymongo
import pandas as pd
import numpy as np
import pprint
import datetime
from datetime import date
import time
from datetime import timedelta, date

In [3]:
pd.set_option('display.max_rows', 500)

In [4]:
# get event data
uri = "mongodb://weather-report-mongodb:2YCR1nxsdoHtvyFJRYwEESBAbkDDmC9QGtnh3cK9zarK5uvWDFCWLpy3jA0vErtr6muGmVpwWz5mQ3TxeEKOOw==@weather-report-mongodb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@weather-report-mongodb@&retrywrites=false"
client = pymongo.MongoClient(uri)
g_cal_db = client["g_cal_data"]
event_container = g_cal_db["events"]

event_list = []
for event in event_container.find():
    start_date_str = event['start']['dateTime']
    start_date_time_obj = datetime.datetime.strptime(start_date_str, '%Y-%m-%dT%H:%M:%S%z')
    
    date_value = start_date_time_obj.date()
    start_time_value = start_date_time_obj.time()
    
    end_date_str = event['end']['dateTime']
    end_date_time_obj = datetime.datetime.strptime(end_date_str, '%Y-%m-%dT%H:%M:%S%z')
    end_time_value = end_date_time_obj.time()
    
    event_list.append(
        pd.DataFrame(
            data={
                'date':[date_value],
                'start':start_time_value,
                'end':end_time_value,
                'start_datetime':(str(date_value) + ' ' + str(start_time_value)),
                'end_datetime':(str(date_value) + ' ' + str(end_time_value))
            },
            dtype=str
        )
    )

event_df = pd.concat(event_list)
event_df['start_datetime'] = pd.to_datetime(event_df['start_datetime'])
event_df['end_datetime'] = pd.to_datetime(event_df['end_datetime'])

In [5]:
event_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 0
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            58 non-null     object        
 1   start           58 non-null     object        
 2   end             58 non-null     object        
 3   start_datetime  58 non-null     datetime64[ns]
 4   end_datetime    58 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 2.7+ KB


In [6]:
# get the current date in yyyy-mm-dd format
#current_date = date.today()

# get the starting time represented as an integer

# initialize list that holds pd records
timestamp_pd_list = []

for date_str in event_df['date'].unique():
    current_date = datetime.datetime.strptime(date_str,'%Y-%m-%d').date()
    start_time = 900
    
    # iterate through and add increments of 15 mins
    for num in range(0,96):
        timestamp_pd_list.append(
            pd.DataFrame(
                data={
                    'datetime':[str(current_date) + ' ' + time.strftime('%H:%M:%S', time.gmtime(start_time))],
                    'is_available':1
                }
            )        
        )
        start_time += 900
timestamp_df = pd.concat(timestamp_pd_list)
timestamp_df['datetime'] = pd.to_datetime(timestamp_df['datetime'])

In [7]:
timestamp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2112 entries, 0 to 0
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   datetime      2112 non-null   datetime64[ns]
 1   is_available  2112 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 49.5 KB


### split out event data between every 15 mins

In [8]:
# get unique start-end times for the given day
unavail_times = event_df[['start_datetime','end_datetime']].drop_duplicates()

In [9]:
# convert these unique events to a list of lists
unavail_times_lol = list(unavail_times.to_records(index=False))

In [10]:
# initialize the available column as 1 (available)
timestamp_df['is_available'] = 1

In [11]:
# mark the calendar as unavailable if an event is detected during that time
for event in unavail_times_lol:
    start = event[0]
    end = event[1]
    timestamp_df.loc[(timestamp_df['datetime'] >= start) & (timestamp_df['datetime'] <= end), 'is_available'] = 0
    #timestamp_df.loc[timestamp_df['is_available'] >= start & timestamp_df['is_available'] <= end, 'is_available'] = 0
    #timestamp_df[(timestamp_df['is_available'] >= start) & (timestamp_df['is_available'] <= end), 'is_available'] = 0
    #timestamp_df['is_available'] = [0 if x >= start and x <= end else 1 for x in timestamp_df['datetime']]

In [26]:
timestamp_df.head(500)

,datetime,is_available
0,2020-08-23 00:15:00,1
0,2020-08-23 00:30:00,1
0,2020-08-23 00:45:00,1
0,2020-08-23 01:00:00,1
0,2020-08-23 01:15:00,1
0,2020-08-23 01:30:00,1
0,2020-08-23 01:45:00,1
0,2020-08-23 02:00:00,1
0,2020-08-23 02:15:00,1
0,2020-08-23 02:30:00,1


### give weather data the same treatment

In [12]:
# get weather report data
wrdb = client["weather_report"]
cont1 = wrdb["daily_temps"]

In [13]:
we_ro_test = cont1.find_one({'date':'2020-09-05'})

In [14]:
weather_hours_list = []
for date_str in event_df['date'].unique():
    current_date = datetime.datetime.strptime(date_str,'%Y-%m-%d').date()
    we_ro_test = cont1.find_one({'date':str(current_date)})
    
    if we_ro_test is not None:
        periods = we_ro_test['periods']
        for period in periods:
            weather_hours_list.append(
                pd.DataFrame(
                    data={
                        'date':[current_date],
                        'time':period['startTime'][:19],
                        'short_forcast':period['shortForecast'],
                        'temperature':period['temperature']
                    }
                )
            )
weather_hours_df = pd.concat(weather_hours_list)
weather_hours_df

,date,time,short_forcast,temperature
0,2020-08-23,2020-08-23T00:00:00,Mostly Cloudy,74
0,2020-08-23,2020-08-23T01:00:00,Mostly Cloudy,73
0,2020-08-23,2020-08-23T02:00:00,Mostly Cloudy,72
0,2020-08-23,2020-08-23T03:00:00,Mostly Cloudy,72
0,2020-08-23,2020-08-23T04:00:00,Mostly Cloudy,71
...,...,...,...,...
0,2020-09-17,2020-09-17T19:00:00,Chance Rain Showers,71
0,2020-09-17,2020-09-17T20:00:00,Chance Rain Showers,71
0,2020-09-17,2020-09-17T21:00:00,Chance Rain Showers,69
0,2020-09-17,2020-09-17T22:00:00,Chance Rain Showers,68


In [22]:
weather_hours_df['time']=pd.to_datetime(weather_hours_df['time'])
weather_hours_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504 entries, 0 to 0
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           504 non-null    object        
 1   time           504 non-null    datetime64[ns]
 2   short_forcast  504 non-null    object        
 3   temperature    504 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 19.7+ KB


In [19]:
# initialize list that holds pd records
timestamp_pd_list_w = []

for date_str in weather_hours_df['date'].unique():
    current_date = date_str #datetime.datetime.strptime(date_str,'%Y-%m-%d').date()

    # get the starting time represented as an integer
    start_time = 900
    
    # iterate through and add increments of 15 mins
    for num in range(0,96):
        timestamp_pd_list_w.append(
            pd.DataFrame(
                data={
                    'datetime':[str(current_date) + ' ' + time.strftime('%H:%M:%S', time.gmtime(start_time))]
                }
            )        
        )
    #     timestamp_pd_list.append(str(current_date) + ' ' + time.strftime('%H:%M:%S', time.gmtime(start_time)))

        start_time += 900
timestamp_df_w = pd.concat(timestamp_pd_list_w)
timestamp_df_w['datetime'] = pd.to_datetime(timestamp_df_w['datetime'])

In [23]:
# round down to the nearest hour
timestamp_df_w['nearest_hour'] = timestamp_df_w['datetime'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 60)))

In [24]:
# try to join the two dataframes
weather_data_pre_dropped = timestamp_df_w.merge(weather_hours_df, left_on='nearest_hour', right_on='time')

In [25]:
weather_data_ready = weather_data_pre_dropped.drop(columns=['nearest_hour', 'date','time'])
weather_data_ready.head()

,datetime,short_forcast,temperature
0,2020-08-23 00:15:00,Mostly Cloudy,74
1,2020-08-23 00:30:00,Mostly Cloudy,74
2,2020-08-23 00:45:00,Mostly Cloudy,74
3,2020-08-23 00:00:00,Mostly Cloudy,74
4,2020-08-23 01:00:00,Mostly Cloudy,73


In [26]:
# join availability with weather data
training_data_to_edit = weather_data_ready.merge(timestamp_df, left_on='datetime', right_on='datetime')
training_data_to_edit.head()

,datetime,short_forcast,temperature,is_available
0,2020-08-23 00:15:00,Mostly Cloudy,74,1
1,2020-08-23 00:30:00,Mostly Cloudy,74,1
2,2020-08-23 00:45:00,Mostly Cloudy,74,1
3,2020-08-23 00:00:00,Mostly Cloudy,74,1
4,2020-08-23 01:00:00,Mostly Cloudy,73,1


In [27]:
training_data_to_edit.to_csv('training_data_no_outcome_var.csv',index=False)

In [28]:
training_data_testing = training_data_to_edit.copy()

In [29]:
training_data_testing['short_forcast'].unique()

array(['Mostly Cloudy', 'Patchy Fog', 'Partly Sunny', 'Mostly Sunny',
       'Partly Cloudy', 'Chance Showers And Thunderstorms', 'Sunny',
       'Mostly Clear', 'Slight Chance Showers And Thunderstorms',
       'Slight Chance Rain Showers', 'Showers And Thunderstorms Likely',
       'Chance Rain Showers', 'Clear'], dtype=object)